In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1,1,28,28)

In [3]:
print(inputs.shape)

torch.Size([1, 1, 28, 28])


# 합성곱층의 풀링 선언하기 

In [5]:
# 1channerl을 입력 받아서 32로 만듭니다, kernel 사이즈는 3이고  딩은 1
conv1 = nn.Conv2d(1,32,3,padding=1)
conv2 = nn.Conv2d(32,64,3,padding=1)
pool = nn.MaxPool2d(2)

In [10]:
out = conv1(inputs)
print(out.shape)
out = conv2(out)
print(out.shape)
out = pool(out)
print(out.shape)
out = pool(out)
print(out.shape)

torch.Size([1, 32, 28, 28])
torch.Size([1, 64, 28, 28])
torch.Size([1, 64, 14, 14])
torch.Size([1, 64, 7, 7])


In [11]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라 
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [12]:
fc = nn.Linear(3136,10)
out = fc(out)

In [14]:
out.shape

torch.Size([1, 10])

In [15]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [16]:
device = 'cuda'
torch.cuda.set_device(1)

In [17]:
learning_rate = 0.001
training_epochs = 15
batch_size= 100

In [18]:
path = '/media/jake/mark-4tb3/input/datasets/'

In [20]:
mnist_train = dsets.MNIST(root=path , # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root=path, # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [21]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                        shuffle= True,
                                          drop_last = True
                                         )

In [34]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32,64 ,kernel_size=3, stride= 1,padding = 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2,stride=2))
        self.fc = torch.nn.Linear(7 * 7 * 64 , 10,bias = True)
        
        torch.nn.init.xavier_uniform_(self.fc.weight)
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out

In [35]:
model = CNN().to(device)

In [36]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [37]:
total_batch = len(data_loader)

In [38]:
total_batch

600

In [39]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y느 ㄴ레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.228291184
[Epoch:    2] cost = 0.0629804581
[Epoch:    3] cost = 0.0461743474
[Epoch:    4] cost = 0.0365707129
[Epoch:    5] cost = 0.0314245373
[Epoch:    6] cost = 0.0254069287
[Epoch:    7] cost = 0.021767715
[Epoch:    8] cost = 0.0189842023
[Epoch:    9] cost = 0.01649471
[Epoch:   10] cost = 0.0144824861
[Epoch:   11] cost = 0.010788694
[Epoch:   12] cost = 0.0102087576
[Epoch:   13] cost = 0.00940682646
[Epoch:   14] cost = 0.0070701465
[Epoch:   15] cost = 0.00696133357
